In [97]:
%load_ext autoreload
%autoreload 2
from __future__ import absolute_import

import torch

import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter

import torch.cuda
import numpy as np

from qpth.qp import QPFunction

import matplotlib.pyplot as plt


class LSTMModel(nn.Module):
    '''
    nn.LSTM Parameters:
        input_size  – The number of expected features in the input x
        hidden_size – The number of features in the hidden state h
        num_layers  – Number of recurrent layers.

    Inputs: input, (h_0, c_0)
        input (seq_len, batch, input_size)
        h_0 (num_layers * num_directions, batch, hidden_size)
        c_0 (num_layers * num_directions, batch, hidden_size)

    Outputs: output, (h_n, c_n)
        output (seq_len, batch, hidden_size * num_directions)
        h_n (num_layers * num_directions, batch, hidden_size)
        c_n (num_layers * num_directions, batch, hidden_size):
    '''
    def __init__(self, inputSize, nHidden, batchSize, noutputs=3, numLayers=2):
        super(LSTMModel, self).__init__()
        
        self.cost = nn.MSELoss(size_average=False)
        self.noutputs = noutputs
        self.num_layers = numLayers
        self.inputSize = inputSize
        self.nHidden = nHidden
        self.batchSize = batchSize
        self.noutputs = noutputs
        
        #define recurrent and linear layers
        self.lstm1  = nn.LSTM(inputSize,nHidden[0],num_layers=numLayers)
        self.lstm2  = nn.LSTM(nHidden[0],nHidden[1],num_layers=numLayers)
        self.lstm3  = nn.LSTM(nHidden[1],nHidden[2],num_layers=numLayers)
        self.fc     = nn.Linear(nHidden[2], noutputs)
        self.drop   = nn.Dropout(0.3)

    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[0])) 
        c0 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[0]))        
        # Forward propagate RNN layer 1
        out, _ = self.lstm1(x, (h0, c0)) 
        out = self.drop(out)
        
        # Set hidden layer 2 states 
        h1 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[1])) 
        c1 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[1]))        
        # Forward propagate RNN layer 2
        out, _ = self.lstm2(out, (h1, c1))  
        
        # Set hidden layer 3 states 
        h2 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[2])) 
        c2 = Variable(torch.Tensor(self.num_layers, batchSize, self.nHidden[2]))        
        # Forward propagate RNN layer 2
        out, _ = self.lstm3(out, (h2, c2)) 
        out = self.drop(out) 
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        return out

#hyperparams
inputSize = 9
nHidden   = [9,6,6]
numLayers = 2
sequence_length = 9
num_epochs = 500
noutputs = 3
batchSize = 1

lstm = LSTMModel(inputSize, nHidden, batchSize, noutputs, numLayers)
#lstm.cuda()

# Loss and Optimizer
criterion = nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(lstm.parameters(), lr=0.1)

#inputs and outputs
for epoch in range(num_epochs):
    inputs = Variable(torch.Tensor(5, 1, 9))
    labels = Variable(torch.Tensor(5, 3))

    # Forward + Backward + Optimize
    optimizer.zero_grad()
    outputs = lstm(inputs)
    loss    = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (epoch % 10) == 0:
        print ('Epoch [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, loss.data[0]))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch [1/500], Loss: nan
Epoch [11/500], Loss: nan
Epoch [21/500], Loss: nan
Epoch [31/500], Loss: nan
Epoch [41/500], Loss: nan
Epoch [51/500], Loss: nan
Epoch [61/500], Loss: nan
Epoch [71/500], Loss: nan
Epoch [81/500], Loss: nan
Epoch [91/500], Loss: nan
Epoch [101/500], Loss: nan
Epoch [111/500], Loss: nan
Epoch [121/500], Loss: nan
Epoch [131/500], Loss: nan
Epoch [141/500], Loss: nan
Epoch [151/500], Loss: nan
Epoch [161/500], Loss: nan
Epoch [171/500], Loss: nan
Epoch [181/500], Loss: nan
Epoch [191/500], Loss: nan
Epoch [201/500], Loss: nan
Epoch [211/500], Loss: nan
Epoch [221/500], Loss: nan
Epoch [231/500], Loss: nan
Epoch [241/500], Loss: nan
Epoch [251/500], Loss: nan
Epoch [261/500], Loss: nan
Epoch [271/500], Loss: nan
Epoch [281/500], Loss: nan
Epoch [291/500], Loss: nan
Epoch [301/500], Loss: nan
Epoch [311/500], Loss: nan
Epoch [321/500], Loss: nan
Epoch [331/500], Loss: nan
Epoch

In [84]:
inputs.cuda()

RuntimeError: cuda runtime error (4) : unspecified launch failure at /home/robotec/Documents/NNs/locuclab-pytorch/torch/lib/THC/generic/THCTensorCopy.c:18

ImportError: No module named rospkg

In [86]:
import sys
help(sys.path.insert)


Help on built-in function insert:

insert(...)
    L.insert(index, object) -- insert object before index



In [88]:
q = Variable(torch.Tensor(3,1))

In [89]:
q

Variable containing:
 5.5608e+31
 4.5883e-41
 1.9190e-09
[torch.FloatTensor of size 3x1]

In [96]:
q.data


 5.5608e+31
 4.5883e-41
 1.9190e-09
[torch.FloatTensor of size 3x1]

In [91]:
f = torch.cuda.is_available()

In [92]:
print f

True
